First we must install the python driver and the image for XTDB

In [ ]:
!pip install 'xtdb==0.0.4.dev0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
# @title {display-mode:"form"}
# @markdown <center><img src="https://xtdb.com/shared/images/logo-text.svg" alt="Logo of XTDB" /></center>
import multiprocessing, time, urllib.request

# >> Utils
def __is_ok(url):
    try:
        with urllib.request.urlopen(url) as response:
            return response.status == 200
    except urllib.error.URLError:
        return False

def __bg(f):
  multiprocessing.Process(target=f).start()

def __container_count():
  out = !udocker --allow-root ps | wc -l
  return int(out[0]) - 1

# >> Install udocker
!pip install --quiet udocker
!udocker --allow-root install

# >> Run XTDB in the background
def __start_xtdb():
  if __container_count() < 1:
    !udocker --allow-root run --publish=3000:3000 ghcr.io/xtdb/xtdb-standalone-ea:latest
__bg(__start_xtdb)

# >> Wait until XTDB is running
while not __is_ok("http://localhost:3000/status"):
  time.sleep(1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 1.3 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.16
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful
1
Info: downloading layer sha256:bcc2588590e744e6702f37133da9bb91d3103af28977690956c3b2c9ba7bc773
Info: downloading layer sha256:a428254436ae8a80527004da13bafee77e38daa8faac99abe6ce05b7f1e9d1a9
Info: downloading layer sha256:e186d819fdccc7f3931c9871b01d3f0b0c01ae01de687378229dbe2934fddc29
Info: downloading layer sha256:827a757d92f9fb1fd8fc05ee55a43799ca5d962023c9fbb7fc7a3c686e9b9c98
Info: downloading layer sha256:88a482a892f797cdf94bf3a78ff38358fc8787db8011ff0735eab1e616e88182
Info: downloading layer sha256:3a4f3b0af088f0ac99bc27258d0ec957d837be8754ad92943679c318bad4411a
Info: downloading layer sha256:71555250e9d75951fa801c00af20690e7a9bcff8db0d671f8d3e8437b8713653
Info: downloading layer sha256:c2d7d89

Everything we need lies in the client and the DB API (PEP 249)

In [ ]:
from xtdb import Xtdb, DBAPI


In [ ]:
!pip install --quiet jupysql
%reload_ext sql
conn = DBAPI().connect("http://localhost:3000")
%sql conn --alias xtdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.7/95.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.0/251.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.0/193.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.5 MB/s eta 0:00:00
Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [ ]:
%sql INSERT INTO docs (xt$id, foo) VALUES (1, 'bar'), (2, 'baz');
foo = 'bar'
%sql SELECT * from docs where docs.foo = '{{foo}}';

Running query in 'xtdb'

Running query in 'xtdb'

{"query": {"sql": "SELECT * from docs where docs.foo = 'bar' "}, "queryOpts": {}}


1 rows affected.

xt/id,foo
1,bar


# Time in Finance

IT systems of all kinds are plagued by many time-related complexities, from misconfigured Network Time Protocol and unexpected certificate expirations, to changing timezones and software concurrency bugs. However, systems in financial domains are additionally accutely affected by the following common requirements and challenges…

## 1) Audit and Replay: What Happened?

SQL databases systems typically lose information whenever UPDATE or DELETE are used. They also don’t record or track the any changes to data by default.

In [ ]:
%%sql
INSERT INTO trades (xt$id, price) VALUES (1, 100);
UPDATE trades SET price = 150 WHERE trades.xt$id = 1;

Running query in 'xtdb'

++
||
++
++

In [ ]:
%sql SELECT trades.xt$id, trades.price FROM trades;

Running query in 'xtdb'

{"query": {"sql": "SELECT trades.xt$id, trades.price FROM trades"}, "queryOpts": {}}


1 rows affected.

price,xt/id
150,1


In common SQL databases, the original price ‘100’ is lost forever, and no information about when it was last changed is recorded. Perhaps if you’re sufficiently desparate and very lucky there might be an old backup of the data available somewhere with a previous value.

As a consequence, software engineers working in financial domains are forced to routinely confront this fundamental ‘mutability’ across their database systems to ensure that sufficient records are kept for satisfying auditors and regulators about exactly how have changed over time.

In XTDB however, there is no need for custom audit tables or timestamp columns to litter the application schema, because a full audit history is maintained automatically, and the history of the entire database can be ‘replayed’ with microsecond granularity.

In [ ]:
%%sql
INSERT INTO trades (xt$id, price) VALUES (1, 100);
UPDATE trades SET price = 150 WHERE trades.xt$id = 1;

Running query in 'xtdb'

++
||
++
++

In [ ]:
%sql SELECT trades.xt$id, trades.price, trades.xt$system_from FROM trades FOR ALL SYSTEM_TIME;

Running query in 'xtdb'

{"query": {"sql": "SELECT trades.xt$id, trades.price, trades.xt$system_from FROM trades FOR ALL SYSTEM_TIME"}, "queryOpts": {}}


4 rows affected.

price,xt/system-from,xt/id
150,2024-05-22 15:39:53.281277,1
100,2024-05-22 15:39:53.260680,1
150,2024-05-22 15:39:52.839110,1
100,2024-05-22 15:39:52.811914,1


XTDB implements the SQL:2011 standard for “system time versioning” ubiquitously, across all tables. Queries can run against any previous database state and have full access to history through the regular SQL interface.

A fully-ordered log of transaction history is maintained:

In [ ]:
%sql SELECT transaction_log.* FROM xt$txs AS transaction_log LIMIT 4;

Running query in 'xtdb'

{"query": {"sql": "SELECT transaction_log.* FROM xt$txs AS transaction_log LIMIT 4"}, "queryOpts": {}}


4 rows affected.

xt/committed?,xt/tx-time,xt/id
True,2024-05-22 15:39:52.811914,1317
True,2024-05-22 15:39:53.281277,5244
True,2024-05-22 15:39:52.839110,2626
True,2024-05-22 15:39:53.260680,3935


# 2) Snapshot-free Reporting: reproducibility without copying and result tearing

Most applications rely on an ability to issue multiple queries against a database, and can’t simply execute a single SQL statement to retrieve all necessary information in a single round trip.

To maintain consistency across multiple queries, databases offer ACID transaction sessions which fully isolate the reading application process from any concurrent writes that may be affecting the records being read.

This session-oriented transaction isolation capability is typically stateful and extremely resource-intensive over long durations due to ‘locking’ of record versions internally within the database, and in turn this degrades the performance of all workloads using the system.

To workaround these limitations of regular transactions, applications that need to run many queries in a consistent manner will often resort to first exporting all the necessary data or using database facilities to create explicit ‘snapshots’ (i.e. redundant copies).

In contrast, XTDB allows applications to refer to previous database states without any declarations ahead of time - queries can simply specify the version of the database (using a ‘basis’ timestamp) and specific tables as needed:


In [ ]:
%%sql
SET import_system_time = '2020-01-01T00:00:00Z';
INSERT INTO trades (xt$id, price) VALUES (1, 100);


Running query in 'xtdb'

++
||
++
++

In [ ]:
%%sql
SET import_system_time = '2020-01-02T00:00:00Z';
INSERT INTO trades (xt$id, price) VALUES (1, 150);


Running query in 'xtdb'

++
||
++
++

In [ ]:
%sql SELECT trades.xt$id, trades.price FROM trades FOR SYSTEM_TIME AS OF DATE '2020-01-01';

Running query in 'xtdb'

{"query": {"sql": "SELECT trades.xt$id, trades.price FROM trades FOR SYSTEM_TIME AS OF DATE '2020-01-01' "}, "queryOpts": {}}


++
||
++
++

No locking, copying, or snapshotting is taking place against within XTDB. Queries against old, stable versions of the data are always consistent and will never contain partial changes or ‘tearing’ in the results.


# 3) Interleaving Upstream Sources: whose timestamp?

In many circumstances the system time of the database itself is uninteresting (i.e outside of auditing / debugging), and what is more relevant is the time recorded in an upstrem system. For instance a High-Frequency Trading platform could generate a trade with a timestamp that is anywhere from 0.01-100ms earlier than the time at which the trade database records the trade. In other situations information may even arrive hours or days ‘late’.

In cases where the upstream timestamps need to be respected for the AS OF application reporting requirement even moreso than the database timestamps, “valid time versioning” is required (implementing SQL:2011’s notion of “application time versioning”).

Crucially, valid-time is suitable for handling out-of-order timestamp information coming from many upstream systems, whereas system-time is always a monotonically increasing local clock. All rows of data in XTDB keep track of system-time and valid-time timestamps, which allows for precise querying.

Similarly to using system-time for snapshot-free reporting, valid-time can be used to run multiple queries consistently against fixed timestamps in conjunction with system-time, creating a “bitemporal timeslice” view of data.


In [ ]:
%%sql
SET import_system_time = '2020-01-02T00:00:00Z';
INSERT INTO trades (xt$id, price, xt$valid_from)
VALUES (2, 200, DATE '2020-01-02');

Running query in 'xtdb'

++
||
++
++

In [ ]:
%%sql
SET import_system_time = '2020-01-03T00:00:00Z';
INSERT INTO trades (xt$id, price, xt$valid_from)
VALUES (1, 100, DATE '2020-01-01');

Running query in 'xtdb'

++
||
++
++

In [ ]:
%%sql
SET import_system_time = '2020-01-04T00:00:00Z';
INSERT INTO trades (xt$id, price, xt$valid_from)
VALUES (4, 400, DATE '2020-01-04');

Running query in 'xtdb'

++
||
++
++

In [ ]:
%%sql
SELECT trades.xt$id, trades.price
  FROM trades FOR VALID_TIME AS OF TIMESTAMP '2020-01-01 00:00:00'
              FOR SYSTEM_TIME AS OF DATE '2020-01-31';

Running query in 'xtdb'

{"query": {"sql": "SELECT trades.xt$id, trades.price\n  FROM trades FOR VALID_TIME AS OF TIMESTAMP '2020-01-01 00:00:00'\n              FOR SYSTEM_TIME AS OF DATE '2020-01-31'"}, "queryOpts": {}}


1 rows affected.

price,xt/id
100,1


In [ ]:
%%sql
SELECT trades.xt$id, trades.price
  FROM trades FOR VALID_TIME AS OF TIMESTAMP '2020-01-04 00:00:00'
              FOR SYSTEM_TIME AS OF DATE '2020-01-31';

Running query in 'xtdb'

{"query": {"sql": "SELECT trades.xt$id, trades.price\n  FROM trades FOR VALID_TIME AS OF TIMESTAMP '2020-01-04 00:00:00'\n              FOR SYSTEM_TIME AS OF DATE '2020-01-31'"}, "queryOpts": {}}


3 rows affected.

price,xt/id
200,2
100,1
400,4


# 4) Complex Version Histories

In addition to storing and reporting against upstream timestamps, financial applications often need to record and display multiple versions of entities.

Within many SQL systems, schema migrations complicate the ability to retain access to prior versions, and many applications resort to storing data as denormalized JSON.

XTDB allows applications to handle diverse shapes of evolving, document-like data natively within regular SQL. No schema is required up-front and regular SQL typing is respected:

In [ ]:
%%sql
SET import_system_time = '2020-01-02T00:00:00Z';
INSERT INTO ts3 (xt$id, info, xt$valid_from) VALUES (1, {'price': 100}, DATE '2020-01-01');
SET import_system_time = '2020-01-03T01:02:03Z';
UPDATE ts3 SET info = {'price': 100, 'qty': 5}
WHERE ts3.xt$id = 1;SET import_system_time = '2020-01-04T00:00:00Z';
UPDATE ts3 SET info = {'price': 101, 'qty': 4, 'model_params':[DATE '2020-06-01', {'gbp_usd': 1.2617}]} WHERE trades.xt$id = 1;


Running query in 'xtdb'

++
||
++
++

In [ ]:
%%sql
SELECT ts.xt$id, ts.info, ts.xt$valid_from, ts.xt$valid_to FROM ts3 FOR VALID_TIME FROM DATE '2020-01-01' TO DATE '2020-01-31' As ts;

Running query in 'xtdb'

{"query": {"sql": "SELECT ts.xt$id, ts.info, ts.xt$valid_from, ts.xt$valid_to FROM ts3 FOR VALID_TIME FROM DATE '2020-01-01' TO DATE '2020-01-31' As ts"}, "queryOpts": {}}


++
||
++
++

# 5) Intelligent Archival: live data and storage tiering with transparent queries

All data has a lifecycle, and in a typical application the window of ‘live’ data is carefully managed to ensure that the existence of old and now-irrelevant data does not impact the ongoing performance of the application more than necessary.

For instance, once a trade has been settled, it no longer needs to be incorporated into new risk calculations.

However, any such 'old' data invariably still needs to be retained beyond any single application's window of interest, so it likely gets migrated of 'archived' as part of a batch processing job into a longer term storage system which has a significantly lower basic operational cost for data retention.

With XTDB's ubiquitous tracking of `system_to` and `valid_to` columns however, manual migration processes become unnecessary. Applications can allow data to be transparently archived into cheaper storage by the database without affecting the performance of working with live data.

Whenever old data is needed to be queried, it can achieved purely via SQL without any out-of-band engineering or processing work.

Applications can set the `valid_to` column explicitly to correspond with data lifecycle archival events like 'settled_at'. Or alternatively they can simple `DELETE` the relevant records (setting `valid_to` to the implicit `CURRENT_TIMESTAMP` at the time of the transaction):


In [ ]:
%%sql
SET import_system_time = '2020-01-02T00:00:00Z';
INSERT INTO ts2 (xt$id, price, xt$valid_from) VALUES (1, 100, DATE '2020-01-01');
SET import_system_time = '2020-01-03T00:00:00Z';
UPDATE ts2 SET price = 105 WHERE ts2.xt$id = 1;
SET import_system_time = '2020-01-10T00:00:00Z';
DELETE FROM ts2 WHERE ts2.xt$id = 1;

In [ ]:
%%sql
SELECT ts2.xt$id, ts2.price, ts2.xt$valid_from, ts2.xt$valid_to FROM ts2 FOR VALID_TIME FROM DATE '2020-01-01' TO DATE '2020-01-31';

Extracting unbounded “change data” from XTDB is therefore trivial, without any additional Change Data Capture complexity or integration work beyond raw SQL.

Only systems with a comprehensive, fine-grained understanding of the data lifecycle can effectively implement storage tiering to reduce long-term costs.